In [ ]:
from IPython.display import display
import ipywidgets as ipw
import widgets
import utils
import os

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

project_selector = widgets.ProjectSelectionWidget()
project_selector.load_dropdown_box()

measurements_selector = widgets.MeasurementSelectionWidget()

analysis_selector = widgets.AnalysisSelectionWidget()

results_widgets = widgets.ObjectPropertiesWidgets("Results")

results_support_files = ipw.FileUpload(multiple = True)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def load_measurements(change):
    if project_selector.dropdown_boxes.children[0].value != -1:
        measurements_selector.load_selector(project_selector.dropdown_boxes.children[0].value)

def create_results_openbis(b):
    results_props = {}
    for prop in CONFIG["objects"]["Results"]["properties"]:
        results_props[prop] = results_widgets.properties_widgets[prop].value
        
    project_permid = project_selector.dropdown_boxes.children[0].value
    project_results_collection = OPENBIS_SESSION.get_collections(project = project_permid, code = "RESULTS_COLLECTION")
    results_parents = []
    results_parents.extend(list(measurements_selector.children[0].value))
    results_parents.extend(list(analysis_selector.children[0].value))
    
    if len(project_results_collection) == 0:
        results_collection = utils.create_openbis_collection(
            OPENBIS_SESSION, 
            project = project_selector.dropdown_boxes.children[0].value, 
            code = "RESULTS_COLLECTION", type = "COLLECTION", props = {"$name": "Results"}
        )
    else:
        results_collection = project_results_collection[0]

    results_object = utils.create_openbis_object(
        OPENBIS_SESSION, type="RESULTS", collection=results_collection, 
        props = results_props, parents = results_parents
    )
    upload_datasets(results_object, results_support_files)
    
    display(utils.Javascript(data = "alert('Upload successful!')"))

def upload_datasets(method_object, support_files_widget):
    for filename in support_files_widget.value:
        file_info = support_files_widget.value[filename]
        utils.save_file(file_info['content'], filename)
        OPENBIS_SESSION.new_dataset(type = 'RAW_DATA', sample = method_object, files = [filename]).save()
        os.remove(filename)

# Create results

## Select project

In [ ]:
display(project_selector.dropdown_boxes)
project_selector.dropdown_boxes.children[0].observe(load_measurements, names = 'value')

## Select analysis

In [ ]:
display(analysis_selector)

## Select measurements

In [ ]:
display(measurements_selector)

## Properties

In [ ]:
display(results_widgets)

## Support files

In [ ]:
display(results_support_files)

## Save results

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
create_button.on_click(create_results_openbis)
quit_button.on_click(close_notebook)